<a id="capsnet_model"></a>
### Capsule Network Model
The Architecture of our CapsNet is very similar to general architecture, except for an addition Capsule Layer.



#### Advantage of Capsule Layer in Text Classification
As you can see, we have used Capsule layer instead of Pooling layer. Capsule Layer eliminates the need for forced pooling layers like MaxPool. In many cases, this is desired because we get translational invariance without losing minute details.

In [32]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from keras.layers import concatenate, GRU, Input, K, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

#### CapsNet parameters

In [33]:
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3

In [34]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

#### Capsule Layer

In [35]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [36]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import string
import emoji

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

import math
import itertools
from collections import Counter
from New_Utils import *
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, LSTM
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.optimizers import *
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import *
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.regularizers import *
import tensorflow as tf
from keras.layers.normalization import BatchNormalization


In [38]:
data_train = pd.read_csv("tweet_emot_train.csv")
print(data_train[0:1000])

                ID  \
0    2017-En-21441   
1    2017-En-31535   
2    2017-En-21068   
3    2017-En-31436   
4    2017-En-22195   
5    2017-En-22190   
6    2017-En-20221   
7    2017-En-22180   
8    2017-En-41344   
9    2017-En-20759   
10   2017-En-40158   
11   2017-En-21744   
12   2017-En-31337   
13   2017-En-10902   
14   2017-En-11274   
15   2017-En-11097   
16   2017-En-11475   
17   2017-En-10989   
18   2017-En-10618   
19   2017-En-40212   
20   2017-En-21011   
21   2017-En-20349   
22   2017-En-40483   
23   2017-En-30807   
24   2017-En-21400   
25   2017-En-11184   
26   2017-En-30230   
27   2017-En-11421   
28   2017-En-21740   
29   2017-En-10966   
..             ...   
970  2017-En-40939   
971  2017-En-20749   
972  2017-En-10495   
973  2017-En-21856   
974  2017-En-31286   
975  2017-En-41417   
976  2017-En-10375   
977  2017-En-40332   
978  2017-En-31312   
979  2017-En-30923   
980  2017-En-20143   
981  2017-En-40754   
982  2017-En-10791   
983  2017-

In [39]:
# We are gonna find what emoticons are used in our dataset
import re
tweets_text = data_train.Tweet.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)

[(248, ': '),
 (131, 'xt'),
 (65, ':)'),
 (29, ':('),
 (7, ';)'),
 (7, ':D'),
 (6, ':/'),
 (3, 'XD'),
 (2, ';-O'),
 (2, ';-)'),
 (2, ':-/'),
 (2, ':-('),
 (1, ':u'),
 (1, ':o'),
 (1, ':c'),
 (1, ":'D")]

In [40]:
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

Happy emoticons: {';)', ';-)', ':)', ':D', 'XD'}
Sad emoticons: {':/', ':('}


In [41]:
def clean_text(text):
    
    ## Remove punctuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    reformed=[]
    HAPPY_SMILEYS= {'XD', ':)', ';)', ';-)', ':D'}
    SAD_SMILEYS={':/', ':('}
    words = text.split()
    for word in words:
        if word in SAD_SMILEYS:
            reformed.append("sad")
        elif word in HAPPY_SMILEYS:
            reformed.append("happy")
        else:
            reformed.append(word)
    text = " ".join(reformed)
    
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split())

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [42]:
data_train['Tweet'] = data_train['Tweet'].map(lambda x: clean_text(x))

In [43]:
pd.set_option('display.max_colwidth', -1)
print(data_train[0:1000])

                ID  \
0    2017-En-21441   
1    2017-En-31535   
2    2017-En-21068   
3    2017-En-31436   
4    2017-En-22195   
5    2017-En-22190   
6    2017-En-20221   
7    2017-En-22180   
8    2017-En-41344   
9    2017-En-20759   
10   2017-En-40158   
11   2017-En-21744   
12   2017-En-31337   
13   2017-En-10902   
14   2017-En-11274   
15   2017-En-11097   
16   2017-En-11475   
17   2017-En-10989   
18   2017-En-10618   
19   2017-En-40212   
20   2017-En-21011   
21   2017-En-20349   
22   2017-En-40483   
23   2017-En-30807   
24   2017-En-21400   
25   2017-En-11184   
26   2017-En-30230   
27   2017-En-11421   
28   2017-En-21740   
29   2017-En-10966   
..             ...   
970  2017-En-40939   
971  2017-En-20749   
972  2017-En-10495   
973  2017-En-21856   
974  2017-En-31286   
975  2017-En-41417   
976  2017-En-10375   
977  2017-En-40332   
978  2017-En-31312   
979  2017-En-30923   
980  2017-En-20143   
981  2017-En-40754   
982  2017-En-10791   
983  2017-

In [44]:
# train_file_path = "../input/donorschooseorg-preprocessed-data/train_preprocessed.csv"
train_file_path = "tweet_emot_train.csv"

# embedding_path = "../input/fastttext-common-crawl/crawl-300d-2M/crawl-300d-2M.vec"
embedding_path = "crawl-300d-2M.vec"

batch_size = 128 # 256
recurrent_units = 16 # 64
dropout_rate = 0.3 
dense_size = 8 # 32
fold_count = 2 # 10

In [45]:
print
train = [s.strip() for s in data_train['Tweet']]

text = train
max1=0

for i in range(0,len(text)):
	data=text[i].split(" ")
	max2=len(data)
	if(max2>max1):
		max1=max2


sequences_length = max1
sentences_length = sequences_length 


In [46]:
data_train['Tweet_label']=data_train['Tweet_label'].str.lower()

train_label=data_train['Tweet_label']
lbl_dict={}
index=0
for dial_lbls in train_label:
	if dial_lbls not in lbl_dict:
		lbl_dict[dial_lbls]=index
		index=index+1

print(lbl_dict)
print(data_train['Tweet_label'])


{'exp': 0, 'sug': 1, 'sta': 2, 'que': 3, 'thr': 4, 'oth': 5, 'req': 6, 'sat': 7, 'sst': 8, 'aue': 9, 'ue': 10, 'ot': 11, 'sa': 12}
0       exp
1       sug
2       exp
3       sug
4       sta
5       sta
6       que
7       exp
8       sta
9       thr
10      sta
11      sug
12      sug
13      sta
14      sta
15      exp
16      sta
17      sta
18      exp
19      sta
20      que
21      thr
22      exp
23      exp
24      exp
25      exp
26      exp
27      sta
28      sta
29      sta
       ... 
6808    oth
6809    oth
6810    que
6811    exp
6812    sug
6813    sta
6814    sta
6815    exp
6816    sta
6817    exp
6818    exp
6819    exp
6820    sta
6821    que
6822    sta
6823    exp
6824    que
6825    exp
6826    exp
6827    que
6828    sug
6829    oth
6830    oth
6831    oth
6832    que
6833    sta
6834    sta
6835    sug
6836    que
6837    sta
Name: Tweet_label, Length: 6838, dtype: object


In [47]:
tokenizer=load_create_tokenizer(data_train['ID'],None,True)
X_train=load_create_padded_data(X_train=train,savetokenizer=False,isPaddingDone=False,maxlen=sequences_length,tokenizer_path='./New_Tokenizer.tkn')
word_index=tokenizer.word_index
embedding_matrix=load_create_embedding_matrix(word_index,len(word_index)+1,300,'glove.6B.300d.txt',False,True,'./Emb_Mat.mat')
f=open("Emb_Mat.mat","rb")
embedding_matrix=pickle.load(f)
f.close()

C:\Users\Sahil Aggarwal\Desktop\Speech Act Classification\New_Utils.py:40: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [48]:
def create_label(label):
	
    Y=[]
    for i in label:
    	xxx=np.empty(int(len(lbl_dict)))
    	xxx.fill(0)
    	j=lbl_dict.get(i)
    	xxx[j]=1
    	Y.append(xxx)
    return Y

label = train_label
Y_train = create_label(label)
print(label)
y_train=np.array([i for i in Y_train])

embedding_dim = 300
print(y_train[1])

0       exp
1       sug
2       exp
3       sug
4       sta
5       sta
6       que
7       exp
8       sta
9       thr
10      sta
11      sug
12      sug
13      sta
14      sta
15      exp
16      sta
17      sta
18      exp
19      sta
20      que
21      thr
22      exp
23      exp
24      exp
25      exp
26      exp
27      sta
28      sta
29      sta
       ... 
6808    oth
6809    oth
6810    que
6811    exp
6812    sug
6813    sta
6814    sta
6815    exp
6816    sta
6817    exp
6818    exp
6819    exp
6820    sta
6821    que
6822    sta
6823    exp
6824    que
6825    exp
6826    exp
6827    que
6828    sug
6829    oth
6830    oth
6831    oth
6832    que
6833    sta
6834    sta
6835    sug
6836    que
6837    sta
Name: Tweet_label, Length: 6838, dtype: object
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [49]:
get_model_func = lambda: get_model(
    embedding_matrix,
    sentences_length,
    dropout_rate,
    recurrent_units,
    dense_size)

In [50]:
X=X_train
y=y_train

In [51]:
lbl_dict={'anger':0 , 'anticipation':1 , 'disgust':2, 'fear':3 , 'love':4 , 'joy':5 , 'sadness':6 , 'surprise':7 , 'trust':8 }
print(lbl_dict)

#Multi-hot encoding
	
Y_train_emo=[]
for i in data_train.index:
    xxx=np.empty(int(len(lbl_dict)))
    xxx.fill(0)
    if(data_train['anger'][i]==1):
        xxx[lbl_dict['anger']]=1      
    if(data_train['anticipation'][i]==1):
        xxx[lbl_dict['anticipation']]=1       
    if(data_train['disgust'][i]==1):
        xxx[lbl_dict['disgust']]=1      
    if(data_train['anticipation'][i]==1):
        xxx[lbl_dict['anticipation']]=1       
    if(data_train['fear'][i]==1):
        xxx[lbl_dict['fear']]=1
    if(data_train['love'][i]==1):
        xxx[lbl_dict['love']]=1
    if(data_train['joy'][i]==1):
        xxx[lbl_dict['joy']]=1
    if(data_train['sadness'][i]==1):
        xxx[lbl_dict['sadness']]=1
    if(data_train['surprise'][i]==1):
        xxx[lbl_dict['surprise']]=1
    if(data_train['trust'][i]==1):
        xxx[lbl_dict['trust']]=1
    Y_train_emo.append(xxx) 
    
y_train_emo=np.array([i for i in Y_train_emo])
print(y_train_emo[1])

{'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'love': 4, 'joy': 5, 'sadness': 6, 'surprise': 7, 'trust': 8}
[0. 0. 0. 0. 1. 1. 0. 0. 0.]


In [55]:
#def get_model(embedding_matrix, sequence_length, dropout_rate, recurrent_units, dense_size):
input1 = Input(shape=(sequences_length,))
embed_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False)(input1)
embed_layer = SpatialDropout1D(rate_drop_dense)(embed_layer)

x = Bidirectional(GRU(gru_len, activation='relu', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(embed_layer)
capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings, share_weights=True)(x)
capsule = Flatten()(capsule)
capsule = Dropout(dropout_p)(capsule)
output = Dense(13, activation='sigmoid',name = 'dialog_acts')(capsule)
output_emo=Dense(9, activation='sigmoid', name = 'emotion')(capsule)
model = Model(inputs=input1, outputs=[output, output_emo])
model.compile(
        loss=['categorical_crossentropy','categorical_crossentropy'],
        optimizer='adam',
        metrics={'dialog_acts':'accuracy'}, loss_weights = [1, 1])

In [56]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 79)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 79, 300)      2052300     input_5[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 79, 300)      0           embedding_5[0][0]                
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 79, 256)      329472      spatial_dropout1d_5[0][0]        
__________________________________________________________________________________________________
capsule_5 

In [57]:
model.fit([X], [y, y_train_emo], epochs=10, batch_size=64, validation_split=0.2)

Train on 5470 samples, validate on 1368 samples
Epoch 1/10
5470/5470 [==============================] - 92s 17ms/step - loss: 5.3513 - dialog_acts_loss: 1.6410 - emotion_loss: 3.7102 - dialog_acts_acc: 0.4223 - val_loss: 5.3420 - val_dialog_acts_loss: 1.6956 - val_emotion_loss: 3.6464 - val_dialog_acts_acc: 0.4020
Epoch 2/10
5470/5470 [==============================] - 84s 15ms/step - loss: 4.9567 - dialog_acts_loss: 1.3007 - emotion_loss: 3.6560 - dialog_acts_acc: 0.4296 - val_loss: 5.3510 - val_dialog_acts_loss: 1.7078 - val_emotion_loss: 3.6432 - val_dialog_acts_acc: 0.4020
Epoch 3/10
5470/5470 [==============================] - 85s 15ms/step - loss: 4.9256 - dialog_acts_loss: 1.2741 - emotion_loss: 3.6515 - dialog_acts_acc: 0.4305 - val_loss: 5.3425 - val_dialog_acts_loss: 1.7008 - val_emotion_loss: 3.6418 - val_dialog_acts_acc: 0.4020
Epoch 4/10
5470/5470 [==============================] - 86s 16ms/step - loss: 4.9108 - dialog_acts_loss: 1.2642 - emotion_loss: 3.6466 - dialog_acts